In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec  2 07:54:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    52W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
#' ' means CPU whereas '/device:G:0' means GPU
# import tensorflow as tf
# tf.test.gpu_device_name()

In [4]:
# tf.__version__

In [5]:
# import time

In [6]:
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()

In [7]:
# !kill -9 -1

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.layers import BatchNormalization
import keras
from keras import backend as K
import matplotlib.pyplot as plt
import sklearn

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import time
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.layers import BatchNormalization
import keras
from keras import backend as K
import matplotlib.pyplot as plt
import sklearn
import numpy as np
import PIL
import cv2
import os

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

In [12]:
classes = ['not fractured', 'fractured']

In [13]:
import os
from sklearn.model_selection import train_test_split
from skimage import io

In [14]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = io.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [15]:
# Replace these paths with the paths to your actual folders
fractured_folder = '/content/drive/My Drive/FracAtlas/images/fractured'
not_fractured_folder = '/content/drive/My Drive/FracAtlas/images/not_fractured'

In [16]:
# Load and preprocess fractured data
x_fractured = []
y_fractured = []

for img_filename in os.listdir(fractured_folder):
    img_path = os.path.join(fractured_folder, img_filename)
    img = load_img(img_path, target_size=(224, 224))  # Adjust target_size as needed
    img = img_to_array(img)
    img = img / 255.0
    x_fractured.append(img)
    y_fractured.append(1)  # Label 1 for fractured

In [17]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [18]:
# Load and preprocess not fractured data
x_not_fractured = []
y_not_fractured = []

for img_filename in os.listdir(not_fractured_folder):
    img_path = os.path.join(not_fractured_folder, img_filename)
    img = load_img(img_path, target_size=(224, 224))  # Adjust target_size as needed
    img = img_to_array(img)
    img = img / 255.0
    x_not_fractured.append(img)
    y_not_fractured.append(0)  # Label 0 for not fractured

In [19]:
X = x_fractured + x_not_fractured
y = y_fractured + y_not_fractured

In [20]:
X = np.array(X)
y = np.array(y)

In [21]:
X.shape

(4093, 224, 224, 3)

In [22]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [23]:
num_samples, height, width, num_channels = x_train.shape
x_train_flattened = x_train.reshape(num_samples, -1)

In [24]:
from imblearn.over_sampling import RandomOverSampler

# Instantiate the RandomOverSampler
ros = RandomOverSampler(random_state=0)

# Resample the training data
x_train_resampled, y_train_resampled = ros.fit_resample(x_train_flattened, y_train)

In [25]:
# Reshape x_train_resampled back to its original shape
x_train_new = x_train_resampled.reshape(-1, height, width, num_channels)

# x_train_resampled_original now has the same shape as x_train


In [26]:
from collections import Counter

print("Class distribution after oversampling:", Counter(y_train_resampled))

Class distribution after oversampling: Counter({0: 2712, 1: 2712})


In [27]:
# Split the training data into train and validation
x_train, x_val, y_train, y_val = train_test_split(x_train_new, y_train_resampled, test_size=0.1, random_state=42)

In [28]:
# One-Hot Encode Labels
num_classes = len(classes)
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

In [29]:
# Assuming y_train_one_hot has shape (num_samples, num_classes)
# Now, find the column indices corresponding to 'fractured' and 'not fractured'
fractured_index = classes.index('fractured')
not_fractured_index = classes.index('not fractured')
print("Index for 'fractured':", fractured_index)
print("Index for 'not fractured':", not_fractured_index)


Index for 'fractured': 1
Index for 'not fractured': 0


In [30]:
# Check the shapes of the datasets
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_val shape:", x_val.shape)
print("y_val shape:", y_val.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (4881, 224, 224, 3)
y_train shape: (4881, 2)
x_val shape: (543, 224, 224, 3)
y_val shape: (543, 2)
x_test shape: (819, 224, 224, 3)
y_test shape: (819, 2)


In [31]:
# Assuming y_test is a NumPy array of shape (600, 2)
class_counts1 = np.sum(y_train, axis=0)
class_counts2 = np.sum(y_test, axis=0)
class_counts3 = np.sum(y_val, axis=0)

In [32]:
class_counts1 #3946 not fractured, 4030 fractured

array([2425., 2456.], dtype=float32)

In [33]:
class_counts2 #240 not fractured, 360 fractured.

array([664., 155.], dtype=float32)

In [34]:
class_counts3

array([287., 256.], dtype=float32)

In [35]:
y_test.dtype

dtype('float32')

In [36]:
from keras.models import load_model
from keras.layers import Lambda
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [38]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import Xception

model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# mark loaded layers as not trainable
for layer in model.layers:
	layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
output = Dense(2, activation='softmax')(flat1)
model = Model(inputs=model.inputs, outputs=output)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)  # Adjust learning rate as needed
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Display the model summary
model.summary()

83683744/83683744 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 111, 111, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  

In [39]:
from tensorflow.keras.callbacks import EarlyStopping

In [40]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=200, callbacks=[early_stopping], verbose=1)
# model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=200, verbose=1)

Epoch 1/50
25/25 [==============================] - 19s 299ms/step - loss: 18.4896 - accuracy: 0.7046 - val_loss: 5.2633 - val_accuracy: 0.8287
Epoch 2/50
25/25 [==============================] - 5s 205ms/step - loss: 3.6716 - accuracy: 0.8543 - val_loss: 2.6131 - val_accuracy: 0.8987
Epoch 3/50
25/25 [==============================] - 5s 205ms/step - loss: 1.3855 - accuracy: 0.9156 - val_loss: 2.5146 - val_accuracy: 0.8766
Epoch 4/50
25/25 [==============================] - 5s 206ms/step - loss: 0.3803 - accuracy: 0.9623 - val_loss: 1.0024 - val_accuracy: 0.9337
Epoch 5/50
25/25 [==============================] - 5s 201ms/step - loss: 0.1823 - accuracy: 0.9771 - val_loss: 1.1664 - val_accuracy: 0.9484
Epoch 6/50
25/25 [==============================] - 5s 201ms/step - loss: 0.1484 - accuracy: 0.9830 - val_loss: 1.8570 - val_accuracy: 0.9116
Epoch 7/50
25/25 [==============================] - 5s 201ms/step - loss: 0.1153 - accuracy: 0.9863 - val_loss: 1.3678 - val_accuracy: 0.9319
Epoc

In [41]:
# !pip install efficientnet

In [42]:
# from efficientnet.tfkeras import EfficientNetB0

In [43]:
# model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# # mark loaded layers as not trainable
# for layer in model.layers:
# 	layer.trainable = False
# # add new classifier layers
# flat1 = Flatten()(model.layers[-1].output)
# #x=Dense(1024,activation='relu')(flat1) # FC layer 1
# #x=Dense(64,activation='relu')(x)   # FC layer 2
# output = Dense(2, activation='softmax')(flat1)
# model = Model(inputs=model.inputs, outputs=output)

# optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5, batch_size=200,verbose=1)


In [44]:
import sklearn
from sklearn.metrics import confusion_matrix

In [45]:
test_loss, test_acc = model.evaluate(np.array(x_test), np.array(y_test), verbose=0)
print(test_acc)
##Evaluating Sensitivity, Accuracy and Kappa scores
y_prob = model.predict(x_test)
Y_pred = y_prob.argmax(axis=-1)

0.8547008633613586
26/26 [==============================] - 2s 31ms/step


In [46]:
y_test_flat = y_test.argmax(axis=-1)
y_test_flat.shape

(819,)

In [47]:
cm1 = confusion_matrix(y_test_flat,Y_pred)
print("confusion matrix \n",cm1)

confusion matrix 
 [[639  25]
 [ 94  61]]


In [48]:
from sklearn.metrics import classification_report

In [49]:
import pandas as pd

In [50]:
print(pd.DataFrame(classification_report(y_test_flat,Y_pred,output_dict=True)).T)
Kappa=sklearn.metrics.cohen_kappa_score(y_test_flat,Y_pred)
print('Kappa=',Kappa)

              precision    recall  f1-score     support
0              0.871760  0.962349  0.914817  664.000000
1              0.709302  0.393548  0.506224  155.000000
accuracy       0.854701  0.854701  0.854701    0.854701
macro avg      0.790531  0.677949  0.710521  819.000000
weighted avg   0.841014  0.854701  0.837489  819.000000
Kappa= 0.42911450980851584


In [51]:
model.save('/content/drive/My Drive/bonedata/Xception_original_fracatlas.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Just conversion, no quantization

In [52]:
from google.colab import files

In [53]:
#convert tf.Keras to tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
mobilenet_noquant_model = converter.convert()

In [54]:
#write the model to a tflite file as a binary file
mobilenet_noquant_tflite = "/content/drive/My Drive/bonedata" + "/Xception_noquant_fracatlas.tflite"
with open(mobilenet_noquant_tflite, "wb") as f:
  f.write(mobilenet_noquant_model)
  # tflite_model1.save(mobilenet_noquant_file)
  # files.download(mobilenet_noquant_file)

# Run the TFLite Models

Now we'll run inferences using the TensorFlow Lite Interpreter to compare the model accuracies.

First, we need a function that runs inference with a given model and images, and then returns the predictions:

In [55]:
mobilenet_noquant_tflite = "/content/drive/My Drive/bonedata" + "/Xception_noquant_fracatlas.tflite"

In [56]:
# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
  global x_test

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices), 2), dtype=np.float32)

  start_time = time.time()
  for i, test_image_index in enumerate(test_image_indices):
    test_image = x_test[test_image_index]
    test_label = y_test[test_image_index] #has dimension of 2

    # Check if the input type is quantized, then rescale input data to uint8
    # if input_details['dtype'] == np.uint8:
    #   input_scale, input_zero_point = input_details["quantization"]
    #   test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output
  end_time = time.time()
  latency = (end_time - start_time) / len(test_image_indices)

  return [predictions, latency]

Evaluate the models on all images

In [57]:
# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global x_test
  global y_test

  test_image_indices = range(x_test.shape[0])
  predictions, latency = run_tflite_model(tflite_file, test_image_indices)
  # print(predictions)
  true_class_indices = np.argmax(y_test, axis=-1)
  predicted_class_indices = np.argmax(predictions, axis=-1)

  accuracy = (np.sum(true_class_indices == predicted_class_indices) * 100) / len(x_test)

  print(pd.DataFrame(classification_report(true_class_indices,predicted_class_indices,output_dict=True)).T)
  Kappa=sklearn.metrics.cohen_kappa_score(true_class_indices,predicted_class_indices)
  print('Kappa=',Kappa)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (model_type, accuracy, len(x_test)))
  print('%s model Latency: %.2f ms per inference' % (model_type, latency*1000))
  cm1 = confusion_matrix(true_class_indices, predicted_class_indices)
  print("confusion matrix \n",cm1)

Evaluate the float32 model:

In [58]:
evaluate_model(mobilenet_noquant_tflite, model_type="Float32")

              precision    recall  f1-score     support
0              0.871760  0.962349  0.914817  664.000000
1              0.709302  0.393548  0.506224  155.000000
accuracy       0.854701  0.854701  0.854701    0.854701
macro avg      0.790531  0.677949  0.710521  819.000000
weighted avg   0.841014  0.854701  0.837489  819.000000
Kappa= 0.42911450980851584
Float32 model accuracy is 85.4701% (Number of test samples=819)
Float32 model Latency: 148.98 ms per inference
confusion matrix 
 [[639  25]
 [ 94  61]]


In [59]:
# Size comparison
import os
non_quantized_size = os.path.getsize('/content/drive/My Drive/bonedata/Xception_noquant_fracatlas.tflite')

print(f'Non-Quantized Model Size: {non_quantized_size} bytes')

Non-Quantized Model Size: 83975084 bytes
